### Generate main data from external files
The file loads, processes and combines the external data files to form the main dataset

In [1]:
#Preliminaries
import pandas as pd
import numpy as np
import os

#The code has the following sections: 
    # Generate main table
    # Weather for each match
    # Elo ratings

###############################################################################
###############################################################################
###############################################################################
### GENERATE MAIN TABLE
match = pd.read_csv(os.getcwd() + '\\raw_data\\European_soccer\\Match.csv', index_col=0) #Load CSV file retrieved from SQLite database

#main_df = match.iloc[:1000,0:8] #TEMPORARY: Work with top 1000 rows for speed. 
main_df = match.iloc[:,0:8] #First columns are various ID columns we might need. 


###############################################################################
### THE ENDOGENOUS SIDE OF THINGS
main_df['draw'] = (match['home_team_goal']==match['away_team_goal']) #Dummy for ties
main_df['home'] = (match['home_team_goal']>match['away_team_goal']) #Dummy for ties
main_df['away'] = (match['home_team_goal']<match['away_team_goal']) #Dummy for ties

#Compute average odds 
main_df['odds_draw'] = match[['B365D','BWD','IWD','LBD','PSD','WHD','SJD','VCD','GBD','BSD']].max(axis=1,skipna=True)
main_df['odds_home'] = match[['B365H','BWH','IWH','LBH','PSH','WHH','SJH','VCH','GBH','BSH']].max(axis=1,skipna=True)
main_df['odds_away'] = match[['B365A','BWA','IWA','LBA','PSA','WHA','SJA','VCA','GBA','BSA']].max(axis=1,skipna=True)


#Compute average odds-probabilities
def prob_from_odds(odds): 
    prob = (1/odds) #lol
    return prob

main_df['odds_prob_draw'] = prob_from_odds(main_df['odds_draw'])
main_df['odds_prob_home'] = prob_from_odds(main_df['odds_home'])
main_df['odds_prob_away'] = prob_from_odds(main_df['odds_away'])

#print('Share of matches ending in a draw: ', round(main_df['draw'].mean(),2))
#print('Average odds probability of a draw:', round(main_df['odds_prob_draw'].mean(),2))

###############################################################################
### THE EXOGENOUS SIDE OF THINGS

### CONVERT ID VARIABLES TO NICELY READABLE STUFF FOR BOTH HUMANS AND MACHINES
teams =  pd.read_csv(os.getcwd() + '\\raw_data\\European_soccer\\Team.csv', index_col='team_api_id') #Index column api
main_df[['team_home','team_home_s']] = main_df.join(teams, on='home_team_api_id')[['team_long_name',	'team_short_name']]
main_df[['team_awat','team_away_s']] = main_df.join(teams, on='away_team_api_id')[['team_long_name',	'team_short_name']]
del teams #Housekeeping

country = pd.read_csv(os.getcwd() + '\\raw_data\\European_soccer\\Country.csv', index_col='id') #Index column api
main_df['country'] = main_df.join(country, on='country_id')[['name']]
del country

league = pd.read_csv(os.getcwd() + '\\raw_data\\European_soccer\\League.csv', index_col='id') #Index column api
main_df['league'] = main_df.join(league[['name']], on='league_id')[['name']]
del league

#Housekeeping: Delete id variables after use
main_df.drop(labels=['country_id', 'league_id','stage','home_team_api_id','away_team_api_id'], axis=1, inplace=True)

### COMPUTE FEATURES BASED ON PREVIOUS GAMES 
main_df.sort_values('date',inplace=True)

#Set parameters for the rolling stuf
rolling_periods = 20
rolling_min = rolling_periods//2 

#Create a temporary dataframe we can play around with. 
match2 = match[['date','match_api_id','home_team_api_id','away_team_api_id','home_team_goal', 'away_team_goal']].sort_values('date')#.groupby('home_team_api_id')


### Aggression and defense seperately for home and away games
# Home aggression: Average goals in last few home games. 
# Compute as rolling average. Unfortunately, this includes current period. Hence, we shift the result one row down. 
match2['home_agg'] = match2.groupby('home_team_api_id')['home_team_goal']\
                    .rolling(rolling_periods, min_periods=rolling_min).mean().shift(1).reset_index(0,drop=True) #reset is technical, in order to return a series. Weird. 
#Unfortunately the  shifting moves the rolling average from the last row to the first. We'll just remove it.                   
match2.loc[match2.groupby('home_team_api_id')['home_agg'].head(1).index, 'home_agg'] = np.nan
#match3 = match2[match2['home_team_api_id']==1957].copy() #Tjek

# Home defensiveness: Average goals on themselves in last few home games. 
match2['home_def'] = match2.groupby('home_team_api_id')['away_team_goal']\
                    .rolling(rolling_periods, min_periods=rolling_min).mean().shift(1).reset_index(0,drop=True) 
match2.loc[match2.groupby('home_team_api_id')['home_def'].head(1).index, 'home_def'] = np.nan
#match3 = match2[match2['home_team_api_id']==2183].copy() #Tjek

# Away aggression: Average goals in last few away games. 
match2['away_agg'] = match2.groupby('away_team_api_id')['away_team_goal']\
                    .rolling(rolling_periods, min_periods=rolling_min).mean().shift(1).reset_index(0,drop=True) 
match2.loc[match2.groupby('away_team_api_id')['away_agg'].head(1).index, 'away_agg'] = np.nan
#match3 = match2[match2['away_team_api_id']==4049].copy() #Tjek

# Away defensiveness: Average goals in last few away games. 
match2['away_def'] = match2.groupby('away_team_api_id')['home_team_goal']\
                    .rolling(rolling_periods, min_periods=rolling_min).mean().shift(1).reset_index(0,drop=True) 
match2.loc[match2.groupby('away_team_api_id')['away_def'].head(1).index, 'away_def'] = np.nan
#match3 = match2[match2['away_team_api_id']==6269].copy() #Tjek


### Aggression and defense pooled for home and away games
#Measures for home team 
match2['home_agg_overall'] = np.nan
match2['home_def_overall'] = np.nan

for team in match2['home_team_api_id'].unique(): 
    #Get a teams goals whether they were away or home
    data = match2.loc[(match2['home_team_api_id'] == team), 
                      ['date', 'home_team_goal', 'away_team_goal']]\
            .rename(columns={ 'home_team_goal': 'own_goals', 'away_team_goal': 'their_goals'})
    data = data.append(match2.loc[(match2['away_team_api_id'] == team), 
                                  ['date', 'away_team_goal', 'home_team_goal']]\
            .rename(columns={ 'away_team_goal': 'own_goals', 'home_team_goal': 'their_goals'}))
    
    #Resort, so matches are in date orer
    data.sort_values(by='date', ascending=True, inplace=True)
    
    #Compute measures based on læst games no matter whether they were away or at home
    data['home_agg_overall'] = data['own_goals']\
                                .rolling(rolling_periods, min_periods=rolling_min).mean().shift(1)
    data['home_def_overall'] = data['their_goals']\
                                .rolling(rolling_periods, min_periods=rolling_min).mean().shift(1)
                                
    match2.loc[(match2['home_team_api_id'] == team), 'home_agg_overall'] = data['home_agg_overall']
    match2.loc[(match2['home_team_api_id'] == team), 'home_def_overall'] = data['home_def_overall']
del team, data

#Measures for away team 
match2['away_agg_overall'] = np.nan
match2['away_def_overall'] = np.nan

for team in match2['away_team_api_id'].unique(): 
    #Get a teams goals whether they were away or home
    data = match2.loc[(match2['away_team_api_id'] == team), 
                      ['date', 'away_team_goal', 'home_team_goal']]\
            .rename(columns={ 'away_team_goal': 'own_goals', 'home_team_goal': 'their_goals'})
    data = data.append(match2.loc[(match2['home_team_api_id'] == team), 
                                  ['date', 'home_team_goal', 'away_team_goal']]\
            .rename(columns={'home_team_goal': 'own_goals', 'away_team_goal': 'their_goals'}))
    
    #Resort, so matches are in date orer
    data.sort_values(by='date', ascending=True, inplace=True)
    
    #Compute measures based on læst games no matter whether they were away or at home
    data['away_agg_overall'] = data['own_goals']\
                                .rolling(rolling_periods, min_periods=rolling_min).mean().shift(1)
    data['away_def_overall'] = data['their_goals']\
                                .rolling(rolling_periods, min_periods=rolling_min).mean().shift(1)
                                
    match2.loc[(match2['away_team_api_id'] == team), 'away_agg_overall'] = data['away_agg_overall']
    match2.loc[(match2['away_team_api_id'] == team), 'away_def_overall'] = data['away_def_overall']
del team, data

#### Home advantage and away disadvantage
#match2['home_adv'] = match2


del rolling_periods, rolling_min

#Lets fold it back into the main dataframe 
match2.drop(labels=['home_team_goal', 'away_team_goal', 'home_team_api_id','away_team_api_id'], axis=1, inplace=True)
main_df = main_df.merge(match2, on=['match_api_id', 'date'], how='left')
del match, match2




In [2]:
###############################################################################
###############################################################################
###############################################################################
### WEATHER FOR EACH MATCH

###############################################################################
### STADIUM COORDINATES
#import r-wikidatar
import requests
url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
query = '''SELECT ?club ?clubLabel ?venue ?venueLabel ?coordinates
WHERE
{
	?club wdt:P31 wd:Q476028 .
	?club wdt:P115 ?venue .
	?venue wdt:P625 ?coordinates .
	SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}'''
data = requests.get(url, params={'query': query, 'format': 'json'}).json()
stadiums = []   
for item in data['results']['bindings']: 
    stadiums.append({
            'club': item['club']['value'],
            'team_home': item['clubLabel']['value'],
            'venue': item['venue']['value'],
            'venueLabel': item['venueLabel']['value'],
            'coordinates': item['coordinates']['value'],
                })
stadiums = pd.DataFrame(stadiums)
del query, url, data, item 
 
import unidecode

def clean_names(input_names): #Input must be pandas series
    vfunc = np.vectorize(unidecode.unidecode)
    output_names =  pd.Series(vfunc(input_names)) #Remove special characters
    output_names = output_names.str.lower() #Lowercase
    output_names = output_names.replace('[\.]','', regex=True) #Remove periods
    output_names = output_names.replace('[\d]','', regex=True) #Remove numbers
    output_names = output_names.replace('[-]',' ', regex=True)
    output_names = output_names.replace('afc','', regex=True)
    output_names = output_names.replace('fc','', regex=True)
    output_names = output_names.replace('uc','', regex=True)
    output_names = output_names.replace('ksv','', regex=True)
    output_names = output_names.replace('club','', regex=True)
#   output_names = output_names.replace('tsv','', regex=True)
#    output_names = output_names.replace('nk','', regex=True)
#    output_names = output_names.replace('sg','', regex=True)
    output_names = output_names.replace('rc','', regex=True)
#    output_names = output_names.replace('osc','', regex=True)
#    output_names = output_names.replace('usl','', regex=True)
#    output_names = output_names.replace('sm','', regex=True)
    output_names = output_names.replace('cd','', regex=True)
#    output_names = output_names.replace('cp','', regex=True)
    #output_names = output_names.replace('[z]','', regex=True)
    #output_names = output_names.replace('cbd','', regex=True)
    
    output_names = output_names.str.strip() #Trim whitespace 
    return output_names

#Manuel adjustments
navn1= [('FCV Dender EH','F.C. Verbroedering Dender Eendracht Hekelgem'),
        ('P. Warszawa','Polonia Warsaw'),
    ('Hertha BSC Berlin','Hertha BSC'),
    ('Sporting Charleroi','R. Charleroi S.C.'),
    ('Sporting Lokeren','K.S.C. Lokeren Oost-Vlaanderen'),
    ('Chievo Verona','A.C. ChievoVerona'),
    ('Athletic Club de Bilbao','Athletic Bilbao B'),
    ('US Boulogne Cote D\'Opale','US Boulogne'),
    ('Xerez Club Deportivo','Xerez C.D.'),
    ('Termalica Bruk-Bet Nieciecza','LKS Nieciecza'), 
    ('Legia Warszawa','Legia Warsaw'),
    ('Cracovia','KS Cracovia'),
    ('DSC Arminia Bielefeld','Arminia Bielefeld'),
    ('LOSC Lille','Lille O.S.C.'),
    ('SM Caen','Stade Malherbe Caen'),
    ('Dundee United','Dundee F.C.'),
    ('Hannover 96','OSV Hannover'),
    ('Hamburger SV','SC Victoria Hamburg'),
    ('Amadora','C.F. Estrela da Amadora'),
    ('Sampdoria','Unione Calcio Sampdoria'),
    ('Udinese','Udinese Calcio'),
    ('Atalanta','Atalanta B.C.'),
    ('Milan','A.C. Milan'),
    ('Roma','A.S. Roma'),
    ('Racing Santander','Racing de Santander'),
    ('CD Numancia','Club Deportivo Numancia'),
    ('Torino','Torino Football Club'),
    ('Catania','Calcio Catania'),
    ('Lecce','Unione Sportiva Lecce'),
    ('Lazio','S.S. Lazio'),
    ('Genoa','Genoa Cricket and Football Club'),
    ('Livorno','Associazione Sportiva Livorno Calcio'),
    ('Parma','Parma Calcio 1913'),
    ('AC Arles-Avignon','Avignon Foot 84'),
    ('Excelsior','SBV Excelsior'),
    ('Cesena','A.C. Cesena'),
    ('Brescia','Brescia Calcio'),
    ('FC Lausanne-Sports','FC Lausanne-Sport'),
    ('FC Augsburg','BC Augsburg'),
    ('Novara','Novara Calcio'),
    ('RC Celta de Vigo','Celta Vigo B'),
    ('Estoril Praia','G.D. Estoril Praia'),
    ('Pescara','Delfino Pescara 1936'),
    ('Sassuolo','U.S. Sassuolo Calcio'),
    ('KV Oostende','A.S.V. Oostende K.M.'),
    ('Watford','Watford Football Club'),
    #('Uniao da Madeira','Uniao Flamengo Santos F.C.'),
    ('Frosinone','Frosinone Calcio'),
    ('FC Basel','FC Concordia Basel'),
]

navn2= [('vitoria guimaraes','vitoria de guimaraes b'),
('standard de liege','standard liege'),
#('sporting cp','sporting e petroleos de cabinda'),
('naval deg de maio','associacao naval o de maio'),
('az','az alkmaar'),
('fiorentina','acf fiorentina'),
('academica de coimbra','associacao academica de coimbra   oaf'),
('vitoria setubal','vitoria'),
('real sporting de gijon','sporting gijon'),
('psv','eindhoven'),
('palermo','societa di calcio palermo'),
('inter','intermierdale de milano'),
('recreativo','recreativo de huelva'),
('villarreal cf','villarreal  de futbol'),
('uniao de leiria, sad','ud leiria'),
('sc beira mar','sc beira mar vale callampa'),
('dundee','dundee football'),
('g ajaccio','gazelec ajaccio'),

]

for navn in navn1: #Manual replacements of some team names 
    stadiums['team_home'].replace(navn[1], navn[0], inplace=True)
del navn, navn1

main_df['funky'] = clean_names(main_df['team_home'])
stadiums['funky'] = clean_names(stadiums['team_home'])

for navn in navn2: #Manual replacements of some team names 
    stadiums['funky'].replace(navn[1], navn[0], inplace=True)
del navn, navn2

stadiums.drop_duplicates(subset=['funky'], keep='first', inplace=True)
stadiums.to_csv(os.getcwd() + '\\work_files\\stadiums.csv')
stadiums.drop(labels=['team_home'], axis=1, inplace=True)

main_df = main_df.merge(stadiums, on='funky', how='left')
main_df2 = main_df.merge(stadiums, on='funky', how='inner')
a = main_df[~main_df['funky'].isin(main_df2['funky'])][['team_home','funky']].drop_duplicates()
a.to_csv(os.getcwd() + '\\work_files\\manglende_hold.csv')
del a, main_df2

main_df.drop(labels=['funky', 'club',  'venue'], axis=1, inplace=True)
del stadiums 



In [3]:
##############################################################################
### WEATHER DATA
#import netCDF4
import xarray as xr
import geopandas as gpd
import gzip
#from shapely.geometry import Point


# https://www.ecad.eu/download/ensembles/downloadchunks.php
with gzip.open(os.getcwd() + '\\raw_data\\Weather\\rr_0.25deg_reg_2011-2017_v17.0.nc.gz') as f: 
    weather_2011 = xr.open_dataset(f)
with gzip.open(os.getcwd() + '\\raw_data\\Weather\\rr_0.25deg_reg_1995-2010_v17.0.nc.gz') as f: 
    weather_1995 = xr.open_dataset(f)    
weather = xr.concat([weather_1995,weather_2011], dim='time')
#weather = weather.to_dataframe()
del weather_2011, weather_1995

#print('Share of nulls:', weather.isnull().sum()/len(weather))
fill_limit = 4 
print('Share of nulls (before changes):', weather.isnull().rr.values.sum()/(weather.isnull().rr.values.sum()+weather.rr.count()))
weather = weather.ffill(dim='latitude', limit=fill_limit)
print('Share of nulls (ffill latitude):', weather.isnull().rr.values.sum()/(weather.isnull().rr.values.sum()+weather.rr.count()))
weather = weather.ffill(dim='longitude', limit=fill_limit)
print('Share of nulls (ffill longitude)', weather.isnull().rr.values.sum()/(weather.isnull().rr.values.sum()+weather.rr.count()))
weather = weather.bfill(dim='latitude', limit=fill_limit)
print('Share of nulls (bfill latitude):', weather.isnull().rr.values.sum()/(weather.isnull().rr.values.sum()+weather.rr.count()))
weather = weather.bfill(dim='longitude', limit=fill_limit)
print('Share of nulls (bfill longitude)', weather.isnull().rr.values.sum()/(weather.isnull().rr.values.sum()+weather.rr.count()))
del fill_limit

#Tjekker
#date = main_df['date'].str[:10][25000]
#longitude = float(main_df['coordinates'][25000].split(" ")[0].replace('Point(', ''))
#latitude = float(main_df['coordinates'][25000].split(" ")[1].replace(')', ''))
#weather.sel(time=date, latitude=latitude, longitude=longitude, method='nearest')
 
main_df['RR'] = np.nan
#main_df['RR_date'] = np.nan
#main_df['RR_lon'] = np.nan
#main_df['RR_lat'] = np.nan


main_df['date_s'] = main_df['date'].str[:10]
main_df['longitude'], main_df['latitude'] = main_df['coordinates'].str.split(" ").str #.replace('Point(', ''))
main_df['longitude'] = main_df['longitude'].str.replace('Point\(', '').astype(float)
main_df['latitude'] = main_df['latitude'].str.replace('\)', '').astype(float)


for i in range(0,len(main_df)): #Find data in weather database, and fill in
    if np.isnan(main_df['latitude'][i]) == False: 
        wea = weather.sel(time=main_df['date_s'][i], 
                          latitude=main_df['latitude'][i], 
                          longitude=main_df['longitude'][i], method='nearest')
        main_df.loc[i, 'RR'] = wea.rr.values
#        main_df.loc[i, 'RR_date'] = wea.time.values
#        main_df.loc[i, 'RR_lat'] = wea.latitude.values
#        main_df.loc[i, 'RR_lon'] = wea.longitude.values
    if i % 1000 == 0: 
        print('Updated for row', i)
del i, weather

#Tjek: Share of nulls. 
print('Share of nulls (RR):', main_df['RR'].isnull().sum()/len(main_df['RR']))

##Tjek: Passer datoerne?
#print('Share of wrong dates (RR):', np.mean(main_df['date_s']!=main_df['RR_date'].str[:10]))
#tjek = main_df[(main_df['date_s']!=main_df['RR_date'].str[:10])]
#main_df.drop(labels=['RR_date', 'RR_lat', 'RR_lon'], axis=1, inplace=True)

main_df.drop(labels=['date_s', 'longitude', 'latitude'], axis=1, inplace=True)
#main_df.drop(labels=['Unnamed: 0'], axis=1, inplace=True)




Share of nulls (before changes): <xarray.DataArray 'rr' ()>
array(0.713535)
Share of nulls (ffill latitude): <xarray.DataArray 'rr' ()>
array(0.668724)
Share of nulls (ffill longitude) <xarray.DataArray 'rr' ()>
array(0.637891)
Share of nulls (bfill latitude): <xarray.DataArray 'rr' ()>
array(0.605013)
Share of nulls (bfill longitude) <xarray.DataArray 'rr' ()>
array(0.58326)
Updated for row 0
Updated for row 1000
Updated for row 2000
Updated for row 3000
Updated for row 4000
Updated for row 5000
Updated for row 6000
Updated for row 7000
Updated for row 8000
Updated for row 9000
Updated for row 10000
Updated for row 11000
Updated for row 12000
Updated for row 13000
Updated for row 14000
Updated for row 15000
Updated for row 16000
Updated for row 17000
Updated for row 18000
Updated for row 19000
Updated for row 20000
Updated for row 21000
Updated for row 22000
Updated for row 23000
Updated for row 24000
Updated for row 25000
Share of nulls (RR): 0.08021863813079795


In [4]:
###############################################################################
###############################################################################
###############################################################################
### ELO RATINGS

# Get the team names used in elo site
teams =  pd.read_csv(os.getcwd() + '\\raw_data\\European_soccer\\Team.csv', index_col='team_api_id') #Index column api
teams_elo = pd.read_csv(os.getcwd() + '\\work_files\\Elo_ratings\\Team_match.csv', encoding='latin-1')
teams_elo.columns = ['elo_name', 'team_long_name']

import unidecode 
def clean_names2(input_names): #Input must be pandas series
    input_names = input_names.replace('[ü]', 'ue', regex=True)
    input_names = input_names.replace('[ö]', 'oe', regex=True)
    input_names = input_names.replace('[ä]', 'ae', regex=True)
    input_names = input_names.replace('[.]', '', regex=True)
    vfunc = np.vectorize(unidecode.unidecode)
    output_names =  pd.Series(vfunc(input_names)) #Remove special characters
    output_names = output_names.str.lower() #Lowercase
    output_names = output_names.replace(' ','', regex=True) #Remove space inbetween words
    return output_names

teams_elo.elo_name = clean_names2(teams_elo.elo_name)
teams_elo['elo_name'].replace('realsociedad', 'sociedad', inplace=True)

teams = teams.merge(teams_elo, how='left', on='team_long_name')
#Tjek
print('Share of nulls (team):', round(teams['elo_name'].isnull().sum()/len(teams['team_long_name']),3))
#teams['elo_name'].isnull().sum()

elo_rating = pd.read_csv(os.getcwd() + '\\raw_data\\Elo_ratings\\elo_rating.csv', index_col=0)
elo_rating.reset_index(inplace=True, drop=True)
elo_rating['Name'] = clean_names2(elo_rating.Club)

elo_rating.index = pd.IntervalIndex.from_arrays(elo_rating['From'], elo_rating['To'], closed='both')

main_df['date_s'] = main_df['date'].str[:10]
main_df['ELO_home'] = np.nan
main_df['ELO_away'] = np.nan

counter = 0 
for team_name, elo_name  in zip(teams['team_long_name'], teams['elo_name']): 
    if str(elo_name) != 'nan':         
        team_elo = elo_rating[elo_rating['Name']==elo_name]
        main_df.loc[main_df['team_home']==team_name, 'ELO_home'] = \
            team_elo.iloc[team_elo.index.get_indexer(main_df\
            .loc[main_df['team_home']==team_name, 'date_s']), 4].values
        main_df.loc[main_df['team_awat']==team_name, 'ELO_away'] = \
            team_elo.iloc[team_elo.index.get_indexer(main_df\
            .loc[main_df['team_awat']==team_name, 'date_s']), 4].values
    counter += 1
    if counter % 50 ==0: 
        print('Finished inserting elo for team', counter)
del team_name, elo_name, counter, team_elo

print('Share of nulls (Home ELO):', round(main_df['ELO_home'].isnull().sum()/len(main_df['ELO_home']),2))
print('Share of nulls (Away ELO):', round(main_df['ELO_away'].isnull().sum()/len(main_df['ELO_away']),2))

#main_df['ELO'] = main_df['date_s'].apply(lambda x: elo_rating.iloc[elo_rating.index.get_loc(x)]['Elo'])

#Work
#team_selector = main_df['team_home']=='KRC Genk'
#team_elo = elo_rating[elo_rating['Club']=='Genk']
##team_elo.loc[pd.idx.get_indexer(main_df.loc[main_df['team_home']=='KRC Genk', 'ELO_home'].date_s), 'Elo']
##main_df.loc[team_selector, 'ELO_home'].apply(lambda x: team_elo.iloc[team_elo.index.get_loc(x)]['Elo'])
#main_df.loc[main_df['team_home']=='KRC Genk', 'ELO_home'] = \
#    team_elo.iloc[team_elo.index.get_indexer(main_df\
#    .loc[main_df['team_home']=='KRC Genk', 'date_s']), 4].values

del teams, teams_elo, elo_rating
# main_df.drop(labels=['Unnamed: 0'], axis=1, inplace=True)


Share of nulls (team): 0.298
Finished inserting elo for team 50
Finished inserting elo for team 100
Finished inserting elo for team 150
Finished inserting elo for team 200
Finished inserting elo for team 250
Share of nulls (Home ELO): 0.21
Share of nulls (Away ELO): 0.21


In [5]:

###############################################################################
###############################################################################
###############################################################################
### SAVE FILE
main_df.to_csv('main_df.csv')